In [1]:
import pandas as pd
import numpy as np

In [2]:
def reduce_size(df):
    df['passenger_count'] = df['passenger_count'].astype('int8')
    df['trip_distance'] = df['trip_distance'].astype('int8')
    df['RatecodeID'] = df['RatecodeID'].astype('int8')
    df['PULocationID'] = df['PULocationID'].astype('int16')
    df['DOLocationID'] = df['DOLocationID'].astype('int16')
    df['payment_type'] = df['payment_type'].astype('int8')
    df['fare_amount'] = df['fare_amount'].astype('float32')
    df['mta_tax'] = df['mta_tax'].astype('float32')
    df['tip_amount'] = df['tip_amount'].astype('float32')
    df['tolls_amount'] = df['tolls_amount'].astype('float32')
    df['total_amount'] = df['total_amount'].astype('float32')
    df['other_amounts']= df['extra']+df['improvement_surcharge']+df['congestion_surcharge']+df['airport_fee'].astype('float32')
    df['other_amounts'] =  df['other_amounts'].astype('float32')
    df.drop(columns=['VendorID', 'store_and_fwd_flag', 'improvement_surcharge','extra','congestion_surcharge','airport_fee'], inplace=True)
    return df

In [3]:
df=pd.DataFrame()
base_url="https://storage.googleapis.com/nyc-taxi-data-covid/data/"
df = pd.DataFrame()
for i in range(2019,2022):
        for j in range(1,13):
                url=base_url+str(i)+"/yellow_tripdata_"+str(i)+"-"+f"{j:02}"+".parquet"
                df_new=pd.read_parquet(url)
                ##Data cleaning removing 
                #Cleaning dates
                df_new = df_new[df_new['tpep_pickup_datetime'].dt.year.isin([2019,2020,2021])]
                df_new = df_new[df_new['tpep_dropoff_datetime'].dt.year.isin([2019,2020,2021])]
                #Cleaning passenger count
                df_new = df_new[(df_new['passenger_count'] != 0) & df_new['passenger_count'].notnull()]
                #Cleaning distance & fare amount 
                df_new = df_new[(df_new['fare_amount'] < 250) & (df_new['trip_distance'] >= 0)]
                df_new = df_new[(df_new['trip_distance'] >= 0) & (df_new['trip_distance'] <= 100)]
                df_new=reduce_size(df_new)
                df=pd.concat([df,df_new])
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,mta_tax,tip_amount,tolls_amount,total_amount,other_amounts
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1,1,238,239,1,6.0,0.5,1.47,0.0,11.27,NaN
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1,1,239,238,1,7.0,0.5,1.50,0.0,12.30,NaN
2,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0,1,238,238,1,6.0,0.5,1.00,0.0,10.80,NaN
3,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0,1,238,151,1,5.5,0.5,1.36,0.0,8.16,NaN
4,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0,1,193,193,2,3.5,0.5,0.00,0.0,4.80,NaN


In [4]:
df.memory_usage(index=True).sum() #8967429892

2915690456

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52065901 entries, 0 to 3111996
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        int8          
 3   trip_distance          int8          
 4   RatecodeID             int8          
 5   PULocationID           int16         
 6   DOLocationID           int16         
 7   payment_type           int8          
 8   fare_amount            float32       
 9   mta_tax                float32       
 10  tip_amount             float32       
 11  tolls_amount           float32       
 12  total_amount           float32       
 13  other_amounts          float32       
dtypes: datetime64[ns](2), float32(6), int16(2), int8(4)
memory usage: 2.7 GB


In [6]:
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'])

In [7]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [8]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

#datetime_dim = datetime_dim[(datetime_dim['drop_year'] == 2019) & (datetime_dim['pick_year'] == 2019)]

datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2020-01-01 00:28:15,0,1,1,2020,2,2020-01-01 00:33:03,0,1,1,2020,2
1,1,2020-01-01 00:35:39,0,1,1,2020,2,2020-01-01 00:43:04,0,1,1,2020,2
2,2,2020-01-01 00:47:41,0,1,1,2020,2,2020-01-01 00:53:52,0,1,1,2020,2
3,3,2020-01-01 00:55:23,0,1,1,2020,2,2020-01-01 01:00:14,1,1,1,2020,2
4,4,2020-01-01 00:01:58,0,1,1,2020,2,2020-01-01 00:04:16,0,1,1,2020,2


In [9]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
#passenger_count_dim = passenger_count_dim[(passenger_count_dim['passenger_count'] != 0) & passenger_count_dim['passenger_count'].notnull()]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim = trip_distance_dim[(trip_distance_dim['trip_distance'] >= 0) & (trip_distance_dim['trip_distance'] <= 100)]


In [10]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]


In [11]:
zone_lookup=pd.read_csv("data/taxi_zone_lookup.csv")
pickup_location_dim=df[['PULocationID']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim=pickup_location_dim.merge(zone_lookup,left_on='PULocationID',right_on='LocationID')
pickup_location_dim['pickup_borough']=pickup_location_dim['Borough']
pickup_location_dim['pickup_zone']=pickup_location_dim['Zone']
pickup_location_dim=pickup_location_dim[['pickup_location_id','pickup_borough','pickup_zone']]
pickup_location_dim.head()

,pickup_location_id,pickup_borough,pickup_zone
0,0,Manhattan,Upper West Side North
1,2,Manhattan,Upper West Side North
2,3,Manhattan,Upper West Side North
3,102,Manhattan,Upper West Side North
4,125,Manhattan,Upper West Side North


In [12]:
dropoff_location_dim=df[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim=dropoff_location_dim.merge(zone_lookup,left_on='DOLocationID',right_on='LocationID')
dropoff_location_dim['dropoff_borough']=dropoff_location_dim['Borough']
dropoff_location_dim['dropoff_zone']=dropoff_location_dim['Zone']
dropoff_location_dim=dropoff_location_dim[['dropoff_location_id','dropoff_borough','dropoff_zone']]
dropoff_location_dim.head()

,dropoff_location_id,dropoff_borough,dropoff_zone
0,0,Manhattan,Upper West Side South
1,13,Manhattan,Upper West Side South
2,37,Manhattan,Upper West Side South
3,59,Manhattan,Upper West Side South
4,63,Manhattan,Upper West Side South


In [13]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [14]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'mta_tax', 'tip_amount', 'tolls_amount','other_amounts','total_amount']]

In [15]:
fact_table.head()

,trip_id,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,mta_tax,tip_amount,tolls_amount,other_amounts,total_amount
0,0,0,0,0,0,0,0,0,6.0,0.5,1.47,0.0,NaN,11.27
1,1,1,1,1,1,1,1,1,7.0,0.5,1.50,0.0,NaN,12.30
2,2,2,2,2,2,2,2,2,6.0,0.5,1.00,0.0,NaN,10.80
3,3,3,3,3,3,3,3,3,5.5,0.5,1.36,0.0,NaN,8.16
4,4,4,4,4,4,4,4,4,3.5,0.5,0.00,0.0,NaN,4.80


In [20]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'nyc-taxi-da-389821-a756b51e36a0.json',
)
datetime_dim.to_gbq('nyc_taxi_da.datetime_dim','nyc-taxi-da',if_exists='append')
passenger_count_dim.to_gbq('nyc_taxi_da.passenger_count_dim','nyc-taxi-da',if_exists='append')
trip_distance_dim.to_gbq('nyc_taxi_da.trip_distance_dim','nyc-taxi-da',if_exists='append')
rate_code_dim.to_gbq('nyc_taxi_da.rate_code_dim','nyc-taxi-da',if_exists='append')
pickup_location_dim.to_gbq('nyc_taxi_da.pickup_location_dim','nyc-taxi-da',if_exists='append')
dropoff_location_dim.to_gbq('nyc_taxi_da.dropoff_location_dim','nyc-taxi-da',if_exists='append')
payment_type_dim.to_gbq('nyc_taxi_da.payment_type_dim','nyc-taxi-da',if_exists='append')
fact_table.to_gbq('nyc_taxi_da.fact_table','nyc-taxi-da',if_exists='append')